In [6]:
import pandas as pd
import pickle
import os
import sys
import numpy as np
import cv2
import torch
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from shapely.geometry import Polygon
from skimage import transform
from torch.utils.data import DataLoader
import torch.nn.functional as F
from floortrans.models import get_model
from floortrans.loaders import (
    FloorplanSVG,
    DictToTensor,
    Compose,
    RotateNTurns
)
from floortrans.plotting import (
    segmentation_plot,
    polygons_to_image,
    draw_junction_from_dict,
    discrete_cmap
)
from floortrans.post_prosessing import (
    split_prediction,
    get_polygons,
    split_validation
)
from mpl_toolkits.axes_grid1 import AxesGrid

discrete_cmap()

os.environ['PYTHONPATH'] = '/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git:' + os.environ.get('PYTHONPATH', '')

rot = RotateNTurns() #
room_classes = ["Background", "Outdoor", "Wall", "Kitchen", "Living Room" ,"Bed Room", "Bath", "Entry", "Railing", "Storage", "Garage", "Undefined"]
icon_classes = ["No Icon", "Window", "Door", "Closet", "Electrical Applience" ,"Toilet", "Sink", "Sauna Bench", "Fire Place", "Bathtub", "Chimney"]
room_classes.append("Door")
data_folder = '../data/cubicasa5k/'
data_file = 'test.txt'
normal_set = FloorplanSVG(data_folder, data_file, format='txt', original_size=True)
data_loader = DataLoader(normal_set, batch_size=1, num_workers=0)
data_iter = iter(data_loader)
# Setup Model
model = get_model('hg_furukawa_original', 51)

n_classes = 44
split = [21, 12, 11]

/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/floortrans/plotting.py:591: UserWarning: Trying to register the cmap 'rooms' which already exists.
  cm.register_cmap(cmap=cmap3)
/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/floortrans/plotting.py:597: UserWarning: Trying to register the cmap 'icons' which already exists.
  cm.register_cmap(cmap=cmap3)
/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/floortrans/plotting.py:605: UserWarning: Trying to register the cmap 'rooms_furu' which already exists.
  cm.register_cmap(cmap=cmap3)


In [26]:
floorplan=next(data_iter)

In [27]:
floorplan.keys()

dict_keys(['image', 'label', 'folder', 'heatmaps', 'scale'])

In [28]:
floorplan['label']

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]])

In [29]:
#room classification just based on walls, windows, doors, etc

In [30]:
label=floorplan['label']
label_np=label.data.numpy()[0]
rooms_floorplan=label_np[0]
icons_floorplan=label_np[1]

In [31]:
icons_floorplan

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [13]:
import pickle

# Replace the file path with your own file path
file_path = "/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/dataframes/df_val.pkl"

# Open the file for reading in binary mode
with open(file_path, "rb") as f:
    # Load the serialized data from the file
    df_val = pickle.load(f)

# Now you can use the deserialized object, df_val, as a regular Pandas DataFrame
print(df_val.head())

   Background  Outdoor  Wall  Kitchen  Living Room  Bed Room  Bath  Entry  \
0         0.0      0.0   0.0      0.0          1.0       0.0   0.0    0.0   
1         0.0      0.0   0.0      1.0          0.0       0.0   0.0    1.0   
2         0.0      0.0   0.0      0.0          1.0       0.0   0.0    0.0   
3         0.0      0.0   0.0      0.0          1.0       0.0   0.0    0.0   
4         0.0      0.0   0.0      1.0          1.0       0.0   1.0    1.0   

   Railing  Storage  Garage  Relative Area      Area  \
0      0.0      0.0     0.0       0.497671   41410.0   
1      0.0      0.0     0.0       0.502329   41797.5   
2      0.0      0.0     0.0       0.497671   41410.0   
3      0.0      0.0     0.0       0.166101   42705.0   
4      0.0      0.0     0.0       0.833899  214397.5   

   Average Area of the Room Type  Number of neighboring rooms  
0                       0.293228                          2.0  
1                       0.264512                          2.0  
2       

In [14]:
df_val

,Background,Outdoor,Wall,Kitchen,Living Room,Bed Room,Bath,Entry,Railing,Storage,Garage,Relative Area,Area,Average Area of the Room Type,Number of neighboring rooms
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.497671,41410.0,0.293228,2.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.502329,41797.5,0.264512,2.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.497671,41410.0,0.293228,2.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166101,42705.0,0.308207,4.0
4,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.833899,214397.5,0.567442,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.396451,74106.0,0.261941,4.0
167,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.838873,156805.0,0.554255,4.0
168,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.286833,15517.5,0.199659,2.0
169,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.713167,38582.0,0.487277,2.0


In [15]:
import pickle

# Replace the file path with your own file path
file_path = "/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/dataframes/X_all_val.pkl"

# Open the file for reading in binary mode
with open(file_path, "rb") as f:
    # Load the serialized data from the file
    x = pickle.load(f)

In [19]:
floorplan

{'image': tensor([[[[1.0000, 1.0000, 1.0000,  ..., 0.9686, 0.9686, 0.9686],
           [1.0000, 0.9922, 0.9922,  ..., 0.9843, 0.9843, 0.9843],
           [0.9922, 1.0000, 1.0000,  ..., 0.9922, 0.9922, 0.9922],
           ...,
           [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],
 
          [[0.9843, 0.9843, 0.9843,  ..., 0.9686, 0.9686, 0.9686],
           [0.9843, 0.9765, 0.9765,  ..., 0.9843, 0.9843, 0.9843],
           [0.9765, 0.9843, 0.9843,  ..., 0.9922, 0.9922, 0.9922],
           ...,
           [1.0000, 1.0000, 1.0000,  ..., 0.9765, 0.9843, 0.9843],
           [1.0000, 1.0000, 1.0000,  ..., 0.9765, 0.9765, 0.9765],
           [1.0000, 1.0000, 1.0000,  ..., 0.9765, 0.9765, 0.9765]],
 
          [[0.9294, 0.9294, 0.9294,  ..., 0.9059, 0.9059, 0.9059],
           [0.9294, 0.9216, 0.9216,  ..., 0.9294, 0.9294, 0.9294],
           [0.9

In [20]:
file_path="/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/dataframes/val_modified_1.pkl"
with open(file_path, 'rb') as f:
    train=pickle.load(f)
for index, floorplan in train.items():
    print(floorplan)
    break

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [21]:
train

{0: array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 1: array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 2: array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 3: array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 4: array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 

In [23]:
unique_elements = set()
counter=0
for key in train:
    unique_elements.update(set(train[key].flatten()))
    counter+=1
    if counter==10:
        break

print(unique_elements)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}


In [24]:
room_classes = ["Background", "Outdoor", "Wall", "Kitchen", "Living Room" ,"Bed Room", "Bath", "Entry", "Railing", "Storage", "Garage", "Undefined"]
len(room_classes)


12

for each floorplan in data_iter,

label=floorplan['label']
label_np=label.data.numpy()[0]
rooms_floorplan=label_np[0]
icons_floorplan=label_np[1]

icons_floorplan is set up like this: 

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)
       
where the different values in the image represent a type of icon. respectively 

icon_classes = ["No Icon", "Window", "Door", "Closet", "Electrical Applience" ,"Toilet", "Sink", "Sauna Bench", "Fire Place", "Bathtub", "Chimney"]



ok now train is a dictionary of length 400, same length as data_iter, each value of the key is a modified version of rooms_floorplan for the xth floorplan in data_iter.

this is what train looks like:


{0: array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 1: array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
  
 where the different values in the image represent a room type
 room_classes = ["Background", "Outdoor", "Wall", "Kitchen", "Living Room" ,"Bed Room", "Bath", "Entry", "Railing", "Storage", "Garage", "Undefined"]


i want to use just doors, walls, and windows to create some kind of model where using just those elements, we can predict rooms_floorplan

how can i do this?


In [27]:
#working

import pandas as pd
import pickle
import os
import sys
import numpy as np
import cv2
import torch
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from shapely.geometry import Polygon
from skimage import transform
from torch.utils.data import DataLoader
import torch.nn.functional as F


from floortrans.models import get_model
from floortrans.loaders import (
    FloorplanSVG,
    DictToTensor,
    Compose,
    RotateNTurns
)
from floortrans.plotting import (
    segmentation_plot,
    polygons_to_image,
    draw_junction_from_dict,
    discrete_cmap
)
from floortrans.post_prosessing import (
    split_prediction,
    get_polygons,
    split_validation
)
from mpl_toolkits.axes_grid1 import AxesGrid

discrete_cmap()

os.environ['PYTHONPATH'] = '/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git:' + os.environ.get('PYTHONPATH', '')

##### 
# HELPER FUNCTIONS 
#####

# This function takes an image of a floor plan and a target class as input and returns a binary mask with 1's at the locations where the target class is present in the image
def isolate_class(rooms, CLASS: int):

    # Create a zero-filled numpy array with the same shape as the input image
    template = np.zeros_like(rooms)

    # Get the row and column indices of the pixels where the target class is present
    rows, cols = np.where(rooms == CLASS)

    # Set the corresponding pixels in the template to 1
    template[rows, cols] = 1
    
    # Return the binary mask
    return template

# Define a list of room classes considered as bad and good classes
bad=[0, 1, 2, 8, 11]
good=[3,4,5,6,7,9,10,12]

# This function takes an image and a list of significant nodes (i.e., classes) as input and returns the contours of the rooms, the contours of the doors, and the centroid locations of the significant nodes
def vis_nodes(img, significant_nodes):
    #signficant nodes exclude rooms we don't care about
    nodes = {}
    room_contours={}
    #door_contours={}
    for c in significant_nodes:
        nodes[c] = []
        
        # Initialize an empty list to hold the contours of the rooms of the current class
        room_contours[c] = []
        
        # Get a binary mask with 1's at the locations where the current class is present in the image
        t = isolate_class(img, c)
        
        # Find the contours of the connected components in the binary mask
        contours, _ = cv2.findContours(t.astype(np.uint8), mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)
        
        # Iterate over each contour and add it to the list of room contours and compute the centroid of the contour and add it to the list of node locations
        for s in contours:
            room_contours[c].append(s)
            nodes[c].append(np.squeeze(np.array(s), 1).mean(0))
    template = img.copy()
    
    # Return the room contours, door contours, and node locations
    return(room_contours, room_contours[12], nodes)

# This function takes the contours of the rooms and doors as input and returns the connections between the rooms as a list of pairs of room indices and as a list of pairs of room centroid locations
def get_edges(img, room_contours, door_contours):
    # Initialize empty lists to hold the room connections as indices and as centroid locations
    connections_int = []
    connections_vis = []
    
    # Iterate over each room contour to compare with other room contours
    for i, room1 in enumerate(room_contours):
        # Check that the first room contour has at least 4 points (i.e., is not a line or a point)
        if len(room1) < 4:
            return -1
        
        # Convert room contour to numpy array and create a shapely Polygon object
        room1_arr = np.array(room1).squeeze(1)
        room1_ply = Polygon(room1_arr).buffer(1)
        
        # Iterate over the remaining room contours
        for j, room2 in enumerate(room_contours[i+1:], start=i+1):
            # Check that the second room contour has at least 4 points (i.e., is not a line or a point)
            if len(room2) < 4:
                return -1
            
            # Convert room contour to numpy array and create a shapely Polygon object
            room2_arr = np.array(room2).squeeze(1)
            room2_ply = Polygon(room2_arr).buffer(1)
            
            # Check if the polygons intersect
            if room1_ply.intersects(room2_ply):
                # If the polygons intersect, add the pair of room indices to the list of connections
                connections_int.append([i, j])
                # Add the pair of room centroids to the list of connection locations
                connections_vis.append([room1_arr.mean(0), room2_arr.mean(0)])
                
            else:
                # Iterate over the door contours to check if there is a door between the two rooms
                for door in door_contours:
                    # Convert door contour to numpy array and create a shapely Polygon object
                    door_arr = np.array(door).squeeze(1)
                    door_ply = Polygon(door_arr).buffer(1)
                    
                    # Check if the door intersects both rooms
                    if room1_ply.intersects(door_ply) and room2_ply.intersects(door_ply):
                        # If there is a door between the two rooms, add the pair of room indices to the list of connections
                        connections_int.append([i, j])
                        # Add the pair of room centroids to the list of connection locations
                        connections_vis.append([room1_arr.mean(0), room2_arr.mean(0)])
    
    # Return the list of connections as indices and as centroid locations
    return connections_int, connections_vis


rot = RotateNTurns() #
room_classes = ["Background", "Outdoor", "Wall", "Kitchen", "Living Room" ,"Bed Room", "Bath", "Entry", "Railing", "Storage", "Garage", "Undefined"]
icon_classes = ["No Icon", "Window", "Door", "Closet", "Electrical Applience" ,"Toilet", "Sink", "Sauna Bench", "Fire Place", "Bathtub", "Chimney"]
room_classes.append("Door")
data_folder = '../data/cubicasa5k/'
data_file = 'test.txt'
normal_set = FloorplanSVG(data_folder, data_file, format='txt', original_size=True)
data_loader = DataLoader(normal_set, batch_size=1, num_workers=0)
data_iter = iter(data_loader)
# Setup Model
model = get_model('hg_furukawa_original', 51)

n_classes = 44
split = [21, 12, 11]

# This function takes a file path as input and returns embeddings, embeddings2, and Y.
def process_file(file_path):
    with open(file_path, 'rb') as f:
        file=pickle.load(f)
    embeddings=None
    Y=None
    
    for index, floorplan in file.items():
        if 11 not in set(floorplan.flatten()): #no undefined rooms when training
            icons = normal_set[index]['label'][1].numpy()
            rows, column = np.where(icons == 2)
            try:
                floorplan[rows, column] = 12
            except:
                continue

            # Get the contours of the rooms, doors, and nodes in the floorplan
            rooms, doors, nodes = vis_nodes(floorplan, good)
            
            rc=[]
            areas=[]
            positions=[]
            for k in rooms.keys():
                if k!=12: #not a door
                    rc+=rooms[k]
                    for count, c in enumerate(rooms[k]):
                        area=cv2.contourArea(c)
                        areas.append(area)
                        positions.append(np.array(c).squeeze(1).mean(0).tolist())
            
            try:
                idx, vis = get_edges(floorplan, rc, doors)
            except:
                get_edges(floorplan, rc, doors)==-1
                continue

            if not idx:
                continue

            nodes_lst = []
            
            for k in rooms.keys():
                if k!=12:
                    nodes_lst += ([k] * len(rooms[k]))
            nodes_lst_updated = []
            
            areas_updated=[]
            room_type_areas=[]
            for i in range(len(nodes_lst)):
                edges = set(np.array(idx).flatten())
                if i in edges:
                    nodes_lst_updated.append(nodes_lst[i])
                    areas_updated.append(areas[i])
            nodes_lst=nodes_lst_updated
            areas=areas_updated
            total_area=sum(areas)
            
            #feature #1 relative areas
            relative_areas=np.array(areas)/total_area
            
            node_attrs = {}
            for i, n in enumerate(nodes_lst):
                node_attrs[i] = room_classes[n]
            
            # Create a NetworkX graph object from the edges
            G = nx.Graph(idx)
            
            # Compute the adjacency matrix of the graph
            A = nx.adjacency_matrix(G)
            
            # calculate the degree of each node
            #feature #3 number of adjacent rooms
            degree_list = list(dict(G.degree(G.nodes())).values())
            
            # Convert the node classes to one-hot vectors and concatenate them with the relative areas
            X = F.one_hot(torch.tensor(nodes_lst), 11).numpy()

            if np.isnan(areas).any() or np.isnan(relative_areas).any() or np.isnan(room_type_areas).any():
                continue
                

            X_with_areas=np.hstack(( np.array(relative_areas).reshape(-1, 1),np.array(areas).reshape(-1,1), np.reshape(degree_list, (len(degree_list), 1))))
            
            try:
                X_all=np.concatenate(([X_all, X_with_areas]))
            except:
                X_all=X_with_areas
                
            H=A@X_with_areas
            if embeddings is None:
                embeddings=H
            else:
                embeddings = np.concatenate(([embeddings , H ]), axis=0)
                
            
            if Y is None:
                Y=X
            else:
                Y=np.concatenate(([Y, X]), axis=0)

    room_classes_names = ["Background", "Outdoor", "Wall", "Kitchen", "Living Room" ,"Bed Room", "Bath", "Entry", "Railing", "Storage", "Garage"]
    column_names = ['Relative Area', 'Area',  'Number of neighboring rooms']
    df = pd.DataFrame(data=embeddings, columns=column_names)
    
    return embeddings, Y, df, X_all #X_all is what can be used for the non-graph model.

        
                
            
                    
                    
            
            
        

/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/floortrans/plotting.py:591: UserWarning: Trying to register the cmap 'rooms' which already exists.
  cm.register_cmap(cmap=cmap3)
/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/floortrans/plotting.py:597: UserWarning: Trying to register the cmap 'icons' which already exists.
  cm.register_cmap(cmap=cmap3)
/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/floortrans/plotting.py:605: UserWarning: Trying to register the cmap 'rooms_furu' which already exists.
  cm.register_cmap(cmap=cmap3)
/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/floortrans/plotting.py:611: UserWarning: Trying to register the cmap 'rooms_furu' which already exists.
  cm.register_cmap(cmap=cmap3)


In [28]:
embeddings, Y, df, X_all=process_file("/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/dataframes/val_modified_1.pkl")


In [29]:
df

,Relative Area,Area,Number of neighboring rooms
0,0.512806,41055.5,2.0
1,0.396363,31733.0,3.0
2,0.603637,48327.5,3.0
3,0.165843,13277.5,2.0
4,0.472557,59696.5,2.0
...,...,...,...
180,0.700193,38303.0,2.0
181,0.299807,16400.5,2.0
182,0.573238,174956.0,2.0
183,0.426762,130250.5,2.0


array([5.12805941e-01, 4.10555000e+04, 2.00000000e+00])